First Tokenize full text corpus

In [1]:
import importlib
import tiktoken
print("tiktoken version: ", importlib.metadata.version("tiktoken"))


tiktoken version:  0.9.0


In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text)) #vocab size (our number of tokens)

5145


Let's remove the first 100 tokens and see what happens

In [3]:
enc_sample = encoded_text[100:]

Easiest ways to create input output target pairs for next-word prediction is to have input tokens x and y has the targets which are the inputs shifted by 1
# Example:
x = [1,2,3,4]
y = [2,3,4,5]

if 1,2,3 is input, output should be 4

Context Length determined how many tokens are included in the input
Model is trained to look at sequence of 4 words or tokens
input x is the first 4 tokens eg 1,2,3,4 and target y is next 4 tokens eg 2,3,4,5

In [4]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print("x: " + str(x))
print("y: " + str(y))


x: [5469, 438, 14363, 938]
y: [438, 14363, 938, 4842]


In [5]:
for i in range(1, context_size+1) :
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[5469] ----> 438
[5469, 438] ----> 14363
[5469, 438, 14363] ----> 938
[5469, 438, 14363, 938] ----> 4842


In [6]:
for i in range(1, context_size+1) :
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

wing ----> --
wing-- ----> his
wing--his ---->  last
wing--his last ---->  Chicago


We are going to implement an efficient dataloader to iterate over the dataset and return input and targets as pytorch tensors (multi-dimensional arrays)
Create 2 tensors, input tensor which contains text that LLM sees, and target that contains text LLM should predict
Pytorch dataset and dataloader class
We will use a sliding window approach

Step 1: Tokenize the entire text
Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length
Step 3: Return the total number of rows in the dataset (rows in the output tensor)
Step 4: Return a single row from the dataset

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset) :
    def __init__(self, txt, tokenizer, max_length, stride) : #max_length is context size
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride) : #stride is length of row of tensor, we are doing sliding window
            input_chunk = token_ids[i : i + max_length]
            output_chunk = token_ids[i+1 : i + max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(output_chunk))
    
    def __len__(self) :
        return len(self.input_ids)

    def __getitem__(self, idx) : #will give us given token and the token that should be predicted after it, tells dataloader how input and output should work
        return self.input_ids[idx], self.target_ids[idx]
        





Based on Pytorch dataset class

each row consists of rows of token ids assigned to that chunk of the tensor

Step 1: Initialize Tokenizer

Step 2: Create Dataset

Step 3: drop_last=True drops the last batch in case last batch is shorter than specified batch size during training to prevent loss spikes

Step 4: The number of CPU processed to use for preprocessing

In [22]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0) :
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

batch_size is the number of batches the model processes at once before updating paramaters
Number of workers is parallel processing
Test Dataloader with batch_size 1, and context_size of 4


In [33]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch) #prints input and output pair from tensors

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [34]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


We can see from the above that since the stride is 1 the next input is shifted by only 1, so there would be a lot of computation, 
we can otherwise move on and make the stride larger to make the next set of words inputs.

The batch_size is only 1 so there was only 1 row in the tensor, now lets make the batch_size 8 and see what happens

In [38]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs) #prints input and output pair from tensors
print(targets)

tensor([[   40,   367,  2885,  1464],
        [  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257]])
tensor([[  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026]])


We have to process this whole batch before making the parameter updates, this one has a lot of overlap and overfitting because the stride is only 1, maybe we should just make the stride larger so this doesn't happen